In [ ]:
# Allows you to change source code of modules without reruning everything
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from Random_Forest_module import *

# Data Processing

##### The following data manipulations are based on work in the Pima EDA notebook.  Certain aspects to explore could be feature engineering that was not explored in this part of the project since the main aim was to build a BRAF algorithm rather than to optimize it.  Creating categorical variables based on insulin levels, BMI, and/or age could be explored to improve the alogirthms predictions.

In [ ]:
df_raw = pd.read_csv('diabetes.csv')
df = df_raw.copy()
print(df_raw.shape)
df_raw.head()

In [ ]:
# We fill cells with a value of 0 as NAN

df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [ ]:
# replace the missing values with the median value for that variable

cols = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
for i in cols:
    df[i].fillna((df[i].median()), inplace=True)

In [ ]:
# given the units of insulin in this dataset and our knowledge of reasonable 2-Hour serum insulin (mu U/ml) ranges
# we can either remove these data or set them to be the upper limit of the 85th percentile, which is ~320
# since we don't have much data we suppress these values rather than removing them

Q1 = df.Insulin.quantile(0.15)
Q3 = df.Insulin.quantile(0.85)
IQR = Q3-Q1
ins_lower = Q1 - 1.5*IQR
ins_upper = Q3 + 1.5*IQR

print(ins_upper)
df.loc[df["Insulin"] > ins_upper,"Insulin"] = ins_upper

In [ ]:
# Check that our distributions match those in the EDA notebook

fig, ax = plt.subplots(4,2, figsize=(15,13))
sns.distplot(df.Age, bins = 20, ax=ax[0,0]) 
sns.distplot(df.Pregnancies, bins = 20, ax=ax[0,1]) 
sns.distplot(df.Glucose, bins = 20, ax=ax[1,0]) 
sns.distplot(df.BloodPressure, bins = 20, ax=ax[1,1]) 
sns.distplot(df.SkinThickness, bins = 20, ax=ax[2,0])
sns.distplot(df.Insulin, bins = 20, ax=ax[2,1])
sns.distplot(df.DiabetesPedigreeFunction, bins = 20, ax=ax[3,0]) 
sns.distplot(df.BMI, bins = 20, ax=ax[3,1])

In [ ]:
# split the data into 80% training data and 20% testing data

df_copy = df.copy()
Train_set = df_copy.sample(frac=0.8, random_state=0)
Test_set = df_copy.drop(Train_set.index)

In [ ]:
Train_set.reset_index(drop=True, inplace=True)
Test_set.reset_index(drop=True, inplace=True)

In [ ]:
Train_set.shape, Test_set.shape

In [ ]:
x_cols = Train_set.columns
x_cols = x_cols.drop('Outcome')
y_cols = ['Outcome']

In [ ]:
X_train = Train_set[x_cols]
X_test = Test_set[x_cols]
Y_train = Train_set[y_cols]
Y_test = Test_set[y_cols]

# Define Parameters and Hyperparameters

In [ ]:
# BRAF and RF parameters

S = 100 # total number of trees
p = 0.5 # fraction of trees to use for BRAF
K_NN = 10 # how many nearest neighbors to find for BRAF dataset
base_tree_num = int(S*(1-p)) # how many trees go into the RF with the full dataset
braf_tree_num = S-base_tree_num # how many trees go into the RF with the BRAF dataset

num_feat = 'log2' # use log2 of the number of available features for each tree.  This speeds up computation time.
depth = 10 # how many splits each tree can make (this is the default value)
min_leaf = 3 # minimum amount of features a tree node should have in order to split
k_folds = 10 # number of cross validation folds

# Create BRAF dataframe for training

In [ ]:
# define the majority and minority flags
mincls = 1
majcls = 0

df_BRAF_train = get_BRAF_df(Train_set, y_cols[0], mincls, majcls, K_NN)

In [ ]:
print(df_BRAF_train.shape)
df_BRAF_train.head()

In [ ]:
# split the BRAF training dataframe into X and Y

X_braf_train = df_BRAF_train[x_cols]
Y_braf_train = df_BRAF_train[y_cols]
X_braf_train.shape, Y_braf_train.shape

# Run Random Forest cross validation on the two training datasets

In [ ]:
# Train the CV for both the BRAF model

y_pred_prob_train = get_cv_preds_braf(X_train, 
                                      Y_train, 
                                      X_braf_train, 
                                      Y_braf_train, 
                                      k_folds,
                                      base_tree_num,
                                      braf_tree_num, 
                                      num_feat, 
                                      depth, 
                                      min_leaf)

In [ ]:
# Get metric scores for the combined models

scores_train = get_cv_scores(y_pred_prob_train)
scores_train_mean = np.mean(scores_train, axis=0)

In [ ]:
# Print the metrics for the training dataset

print('Cross validation metrics for the training data are:')
print(f'Precision: {scores_train_mean[0]:.5f}')
print(f'Recall: {scores_train_mean[1]:.5f}')
print(f'AUROC: {scores_train_mean[2]:.5f}')
print(f'AUPRC: {scores_train_mean[3]:.5f}')

In [ ]:
# Create PRC and ROC plots for the CV datasets

plot_PRC(scores_train_mean[4],scores_train_mean[6],'Cross_Validation',scores_train_mean[3])
plot_ROC(scores_train_mean[5],scores_train_mean[4],'Cross_Validation',scores_train_mean[2])

# Random Forest for the test dataset

In [ ]:
# Create BRAF based on the training data using the same parameters as the cross validation

braf = BiasedRandomForest(X_train, 
                          np.array(Y_train), 
                          X_braf_train, 
                          np.array(Y_braf_train), 
                          num_trees=base_tree_num, 
                          num_trees_braf=braf_tree_num, 
                          num_features=num_feat, 
                          depth=depth, 
                          min_leaf=min_leaf)

In [ ]:
# Get predictions and probabilities based on test data sets

test_pred = braf.predict(X_test.values)
test_prob = braf.predict_proba(X_test.values)

In [ ]:
# Compute metrics for the test data

metrics = Metrics(np.array(np.squeeze(Y_test)) ,test_pred, np.array(test_prob)[:,1])

# confusion matrix
cm_test = metrics.compute_confusion_matrix()

# precision and recall
precision_test, recall_test, _ = metrics.calc_precision_recall(cm_test)

# TPR, FPR, and 'precision rate'
TPR_test, FPR_test, prec_test = metrics.calc_roc_prc()

# AUPRC and AUROC
AUPRC_test = metrics.AUC(TPR_test,prec_test)
AUROC_test = metrics.AUC(FPR_test,TPR_test)

In [ ]:
# Print the metrics for the training dataset

print('Metrics for the test data are:')
print(f'Precision: {precision_test:.5f}')
print(f'Recall: {recall_test:.5f}')
print(f'AUROC: {AUROC_test:.5f}')
print(f'AUPRC: {AUPRC_test:.5f}')

In [ ]:
# Create PRC and ROC plots for the test datasets

plot_PRC(TPR_test,prec_test,'Test_Dataset',AUPRC_test)
plot_ROC(FPR_test,TPR_test,'Test_Dataset',AUROC_test)